In [105]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import keras
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [106]:
input_file = "/Users/tsaiflow/Desktop/NN&Sampling/data_upsampled_ddos(unb)_normal(own)/test/upsampled/test_normal_upsampled_80%_ddos_70%.csv"

In [107]:

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = np.asarray(df[name], dtype = np.float).mean()

    if sd is None:
        sd = np.asarray(df[name], dtype = np.float).std()

    df[name] = (np.asarray(df[name], dtype = np.float) - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [108]:
df_test = pd.read_csv(input_file)
print("Read df_test {} rows.".format(len(df_test)))
df_test = df_test.drop(df_test.columns[0], axis=1)
df_test.dropna(inplace=True,axis=1)

Read df_test 47122 rows.


In [109]:
df_test.head(5)

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,total pkt count,total byte count,Label
0,153202.0,3.0,5.0,26.0,11601.0,20.0,0.0,8.666667,10.263203,4380.0,0.0,2320.2,2207.341206,0,4380,1291.888889,1980.710254,8.0,11627.0,DDoS
1,5569540.0,4.0,0.0,24.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.0,0.000000,6,6,6.000000,0.000000,4.0,24.0,DDoS
2,10526334.0,4.0,0.0,24.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.0,0.000000,6,6,6.000000,0.000000,4.0,24.0,DDoS
3,5248859.0,5.0,0.0,30.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.0,0.000000,6,6,6.000000,0.000000,5.0,30.0,DDoS
4,3401909.0,5.0,0.0,30.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.0,0.000000,6,6,6.000000,0.000000,5.0,30.0,DDoS


In [110]:
encode_numeric_zscore(df_test, ' Flow Duration')
encode_numeric_zscore(df_test, ' Total Fwd Packets')
encode_numeric_zscore(df_test, ' Total Backward Packets')
encode_numeric_zscore(df_test, 'Total Length of Fwd Packets')
encode_numeric_zscore(df_test, ' Total Length of Bwd Packets')
encode_numeric_zscore(df_test, ' Fwd Packet Length Max')
encode_numeric_zscore(df_test, ' Fwd Packet Length Min')
encode_numeric_zscore(df_test, ' Fwd Packet Length Mean')
encode_numeric_zscore(df_test, ' Fwd Packet Length Std')
encode_numeric_zscore(df_test, 'Bwd Packet Length Max')
encode_numeric_zscore(df_test, ' Bwd Packet Length Min')
encode_numeric_zscore(df_test, ' Bwd Packet Length Mean')
encode_numeric_zscore(df_test, ' Bwd Packet Length Std')
encode_numeric_zscore(df_test, ' Min Packet Length')
encode_numeric_zscore(df_test, ' Max Packet Length')
encode_numeric_zscore(df_test, ' Packet Length Mean')
encode_numeric_zscore(df_test, ' Packet Length Std')
encode_numeric_zscore(df_test, 'total pkt count')
encode_numeric_zscore(df_test, 'total byte count')

encode_text_index(df_test, ' Label')

array(['DDoS', 'normal'], dtype=object)

In [111]:
x_test, y_test = to_xy(df_test, ' Label')

In [112]:
import keras.backend as K
from keras.models import load_model

K.clear_session()

In [113]:
model = load_model('model_trained_nosampled_sampling@100%.h5')

In [114]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9995967913076694


In [115]:
from sklearn.metrics import classification_report,confusion_matrix
Cm = confusion_matrix(y_eval,pred)
C = np.sum(Cm)
Cm = Cm/C
print('Confusion Matrix:')
print(np.array_str(Cm, precision=4, suppress_small=True))

Confusion Matrix:
[[ 0.6212  0.0003]
 [ 0.0001  0.3784]]


In [116]:
from sklearn.metrics import precision_recall_fscore_support
print('DDos precision & recall & f1_score & support:')
typ = [ "precision", "recall", "f1_score", "support"]
ddos_prfs = precision_recall_fscore_support(y_eval, pred)
ddos_list = [x for x,_ in ddos_prfs]
for f, b in zip(typ, ddos_list):
    print( f, b)

DDos precision & recall & f1_score & support:
precision 0.999863359978
recall 0.999487792385
f1_score 0.999675540907
support 29285


In [117]:
print('Normal precision & recall & f1_score & support:')
typ = [ "precision", "recall", "f1_score", "support"]
ddos_prfs = precision_recall_fscore_support(y_eval, pred)
ddos_list = [_ for x,_ in ddos_prfs]
for f, b in zip(typ, ddos_list):
    print( f, b)

Normal precision & recall & f1_score & support:
precision 0.9991595697
recall 0.999775747043
f1_score 0.999467563402
support 17837
